# Imports

In [41]:
import os, sys, shutil, random, time
import requests
import json
from datetime import datetime

sys.path.append("../")
from InstagramAPI import InstagramAPI

# 0. Configs

In [42]:
TYPE = "guastafriends"

DATE_BEGIN = datetime(2021,1,1,0,0).timestamp()

PATH_BASE                  = os.path.normpath(os.getcwd() + os.sep + os.pardir)
PATH_BASE_DATA             = os.path.join(PATH_BASE, "data")
PATH_1_CREATING_DATASET    = os.path.join(PATH_BASE_DATA, "1-Creating_Dataset")
PATH_FIRST_DATASET         = os.path.join(PATH_1_CREATING_DATASET, "Dataset")
PATH_FIRST_DATASET_TYPE    = os.path.join(PATH_FIRST_DATASET, TYPE)
PATH_2_PREPARING_DATASET   = os.path.join(PATH_BASE_DATA, "2-Preparing_Dataset")
PATH_CLEAN_DATASET         = os.path.join(PATH_2_PREPARING_DATASET, "Clean_Dataset")
PATH_CLEAN_DATASET_TYPE    = os.path.join(PATH_CLEAN_DATASET, TYPE)
PATH_3_PROCESSING_DATASET  = os.path.join(PATH_BASE_DATA, "3-Processing_Dataset")
PATH_4_CLASSIFYING_DATASET = os.path.join(PATH_BASE_DATA, "4-Classifying_Dataset")

RAIO_FOLLOWERS = 3
RAIO_LIKES = 95

BATCH_LABEL_COUNTER = 7500

BATCH_SIZE = 32
CV = 10

print(f"DATE_BEGIN                : {DATE_BEGIN}")
print(f"PATH_BASE                 : {PATH_BASE}")
print(f"PATH_BASE_DATA            : {PATH_BASE_DATA}")
print(f"PATH_1_CREATING_DATASET   : {PATH_1_CREATING_DATASET}")
print(f"PATH_FIRST_DATASET        : {PATH_FIRST_DATASET}")
print(f"PATH_FIRST_DATASET_TYPE   : {PATH_FIRST_DATASET_TYPE}")
print(f"PATH_2_PREPARING_DATASET  : {PATH_2_PREPARING_DATASET}")
print(f"PATH_CLEAN_DATASET        : {PATH_CLEAN_DATASET}")
print(f"PATH_CLEAN_DATASET_TYPE   : {PATH_CLEAN_DATASET_TYPE}")
print(f"PATH_3_PROCESSING_DATASET : {PATH_3_PROCESSING_DATASET}")
print(f"PATH_4_CLASSIFYING_DATASET: {PATH_4_CLASSIFYING_DATASET}")

DATE_BEGIN                : 1609470000.0
PATH_BASE                 : D:\IC
PATH_BASE_DATA            : D:\IC\data
PATH_1_CREATING_DATASET   : D:\IC\data\1-Creating_Dataset
PATH_FIRST_DATASET        : D:\IC\data\1-Creating_Dataset\Dataset
PATH_FIRST_DATASET_TYPE   : D:\IC\data\1-Creating_Dataset\Dataset\guastafriends
PATH_2_PREPARING_DATASET  : D:\IC\data\2-Preparing_Dataset
PATH_CLEAN_DATASET        : D:\IC\data\2-Preparing_Dataset\Clean_Dataset
PATH_CLEAN_DATASET_TYPE   : D:\IC\data\2-Preparing_Dataset\Clean_Dataset\guastafriends
PATH_3_PROCESSING_DATASET : D:\IC\data\3-Processing_Dataset
PATH_4_CLASSIFYING_DATASET: D:\IC\data\4-Classifying_Dataset


In [43]:
def log_print(data):
    print(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {data}", flush=True)

def error_log(acc, log_type, e):
    log_print(f"Failed to process {acc} account: {str(e)}")
    log_print(f"Appending to file {TYPE}_users_error.log")
    error_file = open(os.path.join(PATH_1_CREATING_DATASET, f"{TYPE}_users_error.log"), "a")
    error_file.write(f"{datetime.now().strftime('%Y-%m-%d %H:%M:%S')} - {log_type} - {acc} - {str(e)}")
    error_file.write("\n")
    error_file.close()
    print("**********************************************\n")

# 1. Creating Dataset

## 1.1. Getting Data

In [26]:
with open(os.path.join(PATH_1_CREATING_DATASET, "logins.json")) as json_file:
     file_logins = json.load(json_file)

def get_login(id_login=0, logins=file_logins["Logins"]):
    api = None
    logged = False
    while not logged:      
        log_print("Trying login in account: " + logins[id_login]["username"])
        api = InstagramAPI(logins[id_login]["username"], logins[id_login]["password"])
        logged = api.login()
        if not logged:
            id_login += 1
            if id_login >= len(logins):
                id_login = 0
            time.sleep(3)
    return (api, id_login)
        
def get_accounts():
    file_accounts = open(os.path.join(PATH_1_CREATING_DATASET, f"{TYPE}_users.txt"), 'r')
    if not os.path.exists(os.path.join(PATH_1_CREATING_DATASET, f"{TYPE}_users.bkp")):
        shutil.copyfile(os.path.join(PATH_1_CREATING_DATASET, f"{TYPE}_users.txt"), os.path.join(PATH_1_CREATING_DATASET, f"{TYPE}_users.bkp"))
    list_accounts = file_accounts.readlines()
    file_accounts.close()
    return list_accounts

def create_folder_acc(acc):
    if not os.path.exists(os.path.join(PATH_FIRST_DATASET_TYPE, acc)):
        log_print(f"Creating folder {acc}")
        if not os.path.exists(PATH_FIRST_DATASET_TYPE):
            os.mkdir(PATH_FIRST_DATASET_TYPE)
        os.mkdir(os.path.join(PATH_FIRST_DATASET_TYPE, acc))
        log_print(f"Folder {acc} created")

def create_file_user_info(user):
    log_print("Creating file user_info")
    user["collected_at"] = datetime.timestamp(datetime.now())
    with open(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "user_info.json"), 'w+', encoding="utf-8") as json_file:
        json.dump(user, json_file, indent=4, ensure_ascii=False)
    log_print("File user_info created")

def get_user_feed(api, id_login, user, next_max_id=None):
    get_user_feed = False
    while not get_user_feed:
        if next_max_id == None:
            api.getUserFeed(user['pk'], minTimestamp=int(DATE_BEGIN))
        else:
            api.getUserFeed(user['pk'], maxid=next_max_id, minTimestamp=int(DATE_BEGIN))
        if api.LastResponse.status_code == 429:
            (api, id_login) = get_login(id_login=id_login)
        else:
            get_user_feed = True
    return (api, id_login)

def create_file_user_posts(api, id_login, user):
    log_print("Creating file user_posts")
    (api, id_login) = get_user_feed(api, id_login, user)
    
    if api.LastResponse.status_code == 400:
        raise Exception(api.LastJson)
        
    posts = api.LastJson

    while (posts["more_available"]):
        time.sleep(round(random.uniform(1,3), 1))
        aux_bool = False
        (api, id_login) = get_user_feed(api, id_login, user, posts["next_max_id"])

        posts["items"].extend(api.LastJson["items"])
        posts["num_results"] += api.LastJson["num_results"]
        posts["more_available"] =  api.LastJson["more_available"]
        if posts["more_available"]:
            posts["next_max_id"] = api.LastJson["next_max_id"]
        else:
            del posts['next_max_id']
        posts["auto_load_more_enabled"] = api.LastJson["auto_load_more_enabled"]
        posts["status"] = api.LastJson["status"]
    posts["collected_at"] = datetime.timestamp(datetime.now())
    with open(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "user_posts.json"), 'w+', encoding="utf-8") as json_file:
        json.dump(posts, json_file, indent=4, ensure_ascii=False)
    log_print("File user_posts created")

In [27]:
(api, id_login) = get_login()

log_print("Type: " + TYPE)
log_print("Starting...\n")

list_accounts = get_accounts()
for acc in list_accounts:
    acc = str(acc).strip()

    try:
        log_print(f"Searching account {acc}")
        time.sleep(round(random.uniform(0.5, 1.5), 1))
        api.searchUsername(acc)
        if api.LastResponse.status_code == 200:
            imgs_user = {}
            imgs_user["user"] = acc
            imgs_user["downloaded"] = 0
            
            create_folder_acc(acc)
            user = api.LastJson["user"]
            create_file_user_info(user)
            create_file_user_posts(api, id_login, user)
            
    except Exception as e:
        error_log(acc, "1.1", e)
    else:
        log_print(f"Account {acc} successfully processed")
        print("**********************************************\n")

2021-06-20 21:05:03 - Trying login in account: albert.collector99
Request return 405 error!
{'message': '', 'status': 'fail'}
Request return 404 error!
Login success!

2021-06-20 21:05:06 - Type: guastafriends
2021-06-20 21:05:06 - Starting...

2021-06-20 21:05:06 - Searching account _guasta
2021-06-20 21:05:08 - Creating file user_info
2021-06-20 21:05:08 - File user_info created
2021-06-20 21:05:08 - Creating file user_posts
2021-06-20 21:05:08 - File user_posts created
2021-06-20 21:05:08 - Account _guasta successfully processed
**********************************************

2021-06-20 21:05:08 - Searching account joao.squinelato
2021-06-20 21:05:10 - Creating file user_info
2021-06-20 21:05:10 - File user_info created
2021-06-20 21:05:10 - Creating file user_posts
2021-06-20 21:05:11 - File user_posts created
2021-06-20 21:05:11 - Account joao.squinelato successfully processed
**********************************************

2021-06-20 21:05:11 - Searching account analiviafr
2021-0

## 1.2. Creating Overview

In [33]:
def create_file_user_posts_overview(posts):
    
    log_print("Creating file user_posts_overview")
    
    media_count_total = 0
    posts_overview = {}
    posts_overview["collected_at"] = posts["collected_at"]
    posts_overview["num_results"] = posts["num_results"]
    posts_overview["items"] = []
    
    for post in posts["items"]:
        if post["media_type"] == 1 or post["media_type"] == 8:
            
            post_overview = {}
            post_overview["pk"] = post["pk"]
            post_overview["id"] = post["id"]
            post_overview["taken_at"] = post["taken_at"]
            post_overview["taken_at_year"] = datetime.fromtimestamp(post_overview["taken_at"]).year
            
            post_overview["caption_text"] = ""
            if not post["caption"] is None:
                post_overview["caption_text"] = post["caption"]["text"]
                
            post_overview["media_type"] = post["media_type"]
            post_overview["images"] = []
            if post_overview["media_type"] == 1:
                post_overview["media_count"] = 1
                image = {}
                image["pk"] = post_overview["pk"]
                image["width"] = post["image_versions2"]["candidates"][0]["width"]
                image["height"] = post["image_versions2"]["candidates"][0]["height"]
                image["url"] = post["image_versions2"]["candidates"][0]["url"]
                post_overview["images"].append(image)
            elif post_overview["media_type"] == 8:
                post_overview["media_count"] = post["carousel_media_count"]
                for carousel_post in post["carousel_media"]:
                    image = {}
                    image["pk"] = carousel_post["pk"]
                    image["width"] = carousel_post["image_versions2"]["candidates"][0]["width"]
                    image["height"] = carousel_post["image_versions2"]["candidates"][0]["height"]
                    image["url"] = carousel_post["image_versions2"]["candidates"][0]["url"]
                    post_overview["images"].append(image)
                    
            media_count_total += post_overview["media_count"]
            
            post_overview["location"] = None
            if "location" in post:
                post_overview["location"] = post["location"]

            post_overview["like_count"] = post["like_count"]
            
            post_overview["comments_disabled"] = False
            if "comments_disabled" in post:
                post_overview["comments_disabled"] = post["comments_disabled"]
            if post_overview["comments_disabled"]:
                post_overview["comment_count"] = 0
            else:
                post_overview["comment_count"] = post["comment_count"]

            post_overview["caption_text_length"] = len(post_overview["caption_text"])
            hashtags = [word[1:] for word in post_overview["caption_text"].split() if word[0] == '#']
            post_overview["caption_hashtags_count"] = len(hashtags)
            post_overview["caption_hashtags"] = []
            
            if post_overview["caption_hashtags_count"] > 0:
                post_overview["caption_hashtags"] = hashtags
            post_overview["timestamp_duration"] = posts_overview["collected_at"] - post_overview["taken_at"]
            post_overview["likes_by_duration"] = post_overview["like_count"] / post_overview["timestamp_duration"]
            post_overview["comments_by_duration"] = post_overview["comment_count"] / post_overview["timestamp_duration"]
            
            posts_overview["items"].append(post_overview)
            
    posts_overview["media_count_total"] = media_count_total

    with open(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "user_posts_overview.json"), 'w+', encoding="utf-8") as json_file:
        json.dump(posts_overview, json_file, indent=4, ensure_ascii=False)
    log_print("File user_posts_overview created")

In [36]:
list_accounts = get_accounts()
for acc in list_accounts:
    acc = str(acc).strip()
    
    try:
        log_print(f"Processing account {acc}")
        
        with open(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "user_posts.json"), encoding="utf-8") as json_file:
            posts = json.load(json_file)

        create_file_user_posts_overview(posts)

    except Exception as e:
        error_log(acc, "1.2", e)
    else:
        log_print(f"Account {acc} successfully processed")
        print("**********************************************\n")

2021-06-20 21:19:18 - Processing account _guasta
2021-06-20 21:19:19 - Creating file user_posts_overview
2021-06-20 21:19:19 - File user_posts_overview created
2021-06-20 21:19:19 - Account _guasta successfully processed
**********************************************

2021-06-20 21:19:19 - Processing account joao.squinelato
2021-06-20 21:19:19 - Creating file user_posts_overview
2021-06-20 21:19:19 - File user_posts_overview created
2021-06-20 21:19:19 - Account joao.squinelato successfully processed
**********************************************

2021-06-20 21:19:19 - Processing account analiviafr
2021-06-20 21:19:19 - Creating file user_posts_overview
2021-06-20 21:19:19 - File user_posts_overview created
2021-06-20 21:19:19 - Account analiviafr successfully processed
**********************************************

2021-06-20 21:19:19 - Processing account nmttlbch
2021-06-20 21:19:19 - Failed to process nmttlbch account: [Errno 2] No such file or directory: 'D:\\IC\\data\\1-Creating_

## 1.3. Downloading Images

In [49]:
def download_image(url, path):
    resp = requests.get(url, stream=True, timeout=20)  # Open the url image, set stream to True, this will return the stream content.
    local_file = open(path, 'wb')  # Open a local file with wb ( write binary ) permission.
    resp.raw.decode_content = True  # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
    shutil.copyfileobj(resp.raw, local_file)  # Copy the response stream raw data to local image file.

In [58]:
again = True
while again:
    again = False
    list_accounts = get_accounts()
    for acc in list_accounts:
        acc = str(acc).strip()

        try:
            log_print(f"Processing account {acc}")
            
            if not os.path.exists(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "images")):
                os.mkdir(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "images"))
                
            with open(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "user_posts_overview.json"), encoding="utf-8") as json_file:
                posts_overview = json.load(json_file)

            log_print("Downloading posts pictures:")
            imgs_user_downloaded = len(list(next(os.walk(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "images")))[2])) - 1 #0 - root, 1 - dirs, 2 - files
            if imgs_user_downloaded <= 0:
                imgs_user_downloaded = 0
            if imgs_user_downloaded < posts_overview['media_count_total']:
                for post in posts_overview["items"]:
                    if not "comments_disabled" in post or post["comments_disabled"] is False:
                        for image in post["images"]:
                            if not os.path.exists(os.path.join(PATH_FIRST_DATASET_TYPE, acc, "images", str(image["pk"]) + ".jpg")):
                                download_image(image['url'], os.path.join(PATH_FIRST_DATASET_TYPE, acc, "images", f"{image['pk']}.jpg"))
                                imgs_user_downloaded += 1
                                log_print(f"{imgs_user_downloaded}/{posts_overview['media_count_total']}")
                
            log_print(f"{imgs_user_downloaded} image(s) downloaded")
        except Exception as e:
            error_log(acc, "1.3", e)
            again = True
            time.sleep(3)
        else:
            log_print(f"Account {acc} successfully processed")
            print("**********************************************\n")
print("Finished")

2021-06-20 22:10:55 - Processing account _guasta
2021-06-20 22:10:55 - Failed to process _guasta account: [WinError 3] O sistema não pode encontrar o caminho especificado: 'D:\\IC\\data\\1-Creating_Dataset\\Dataset\\guastafriends\\_guasta\\images'
2021-06-20 22:10:55 - Appending to file guastafriends_users_error.log
**********************************************

2021-06-20 22:10:58 - Processing account joao.squinelato
2021-06-20 22:10:58 - Downloading posts pictures:
2021-06-20 22:10:58 - 9 image(s) downloaded
2021-06-20 22:10:58 - Account joao.squinelato successfully processed
**********************************************

2021-06-20 22:10:58 - Processing account analiviafr
2021-06-20 22:10:58 - Downloading posts pictures:
2021-06-20 22:10:58 - 6 image(s) downloaded
2021-06-20 22:10:58 - Account analiviafr successfully processed
**********************************************

2021-06-20 22:10:58 - Processing account thayfcruz
2021-06-20 22:10:58 - Downloading posts pictures:
2021-06

KeyboardInterrupt: 